In [1]:
import pandas as pd
import numpy as np

**make a lastest date based on the following**
* ARCH,BIO,ASB
* if ASB is POS that lastest date will be CAC date
* if ASB is ND lasest date will be Date received

#### Columns needed:  APN_ROW Segment,	Street #,	Street Name,	Debris Crew

**Asbestos**
* ASB Results
* CAC Clearance
* ASB Results Received

**ARC**
* Arch Assessment

**BIO**
* Bio Assessment





# Steps:
* 1) Compare all dates to Derbirs removal date. DRd should **match** the latest date.
* 2) see if other date are greater. If so print out that date
* 3) see if DRD is equal to anyone orther dates
* 4) if all dates are 0 then leave blank

In [2]:
df = pd.read_excel('../Central Branch  trackers/Central Division 2021 Debris Removal Tracker.xlsx')

In [3]:
# df.info(verbose=True)

In [4]:
# create new columns for each area we might have to check
checks = df[['APN_ROW Segment', 'Street #', 'Street Name', 'Debris Crew', 'Structural Status',
             'ASB Results', 'CAC Clearance', 'ASB Results Received',
            'Arch Assessment',
            'Bio Assessment',
            'Ready for Debris Removal']]

In [5]:
# set up which structal status to not look for
checks = checks[(~checks['Structural Status'].isin(['Site Assessment Complete','Chimney Tip Needed',
                                                    'Asbestos Abatement Needed', 'ROE Received','Pending Asbestos Results']))]

# Change datetimes to string and fill NAs with "0"s

In [6]:
checks = checks.copy()

In [7]:
checks['CAC Clearance'] = checks['CAC Clearance'].dt.strftime('%Y-%m-%d').fillna("0")
checks['ASB Results Received'] = checks['ASB Results Received'].dt.strftime('%Y-%m-%d').fillna("0")
checks['Arch Assessment'] = checks['Arch Assessment'].dt.strftime('%Y-%m-%d').fillna("0")
checks['Bio Assessment'] = checks['Bio Assessment'].dt.strftime('%Y-%m-%d').fillna("0")
checks['Ready for Debris Removal'] = checks['Ready for Debris Removal'].dt.strftime('%Y-%m-%d').fillna('0')

In [8]:
# checks_style = checks.style.highlight_max(color = 'yellow', axis=0, subset=['CAC Clearance','ASB Results Received',
#                                                                             'Arch Assessment', 'Bio Assessment'])

In [9]:
# checks_style

In [10]:
# checks_style.to_excel('Test style date.xlsx')

In [11]:
def find_lastest_date(df, comp1, comp2, comp3, comp4, comp5):
    # Debris not finished
    if df[comp1] == "0":
        return 'No "Ready for Debris Removal" Date'
    
    
    
    # compares comp1 to all other comps
    if ((df[comp1] > df[comp2]) & (df[comp1] > df[comp3]) & (df[comp1] > df[comp4]) & (df[comp1] > df[comp5])):
        return '"Ready for Debris Removal" is greater than Latest Date'
    # compares comp2 to all other comps
    elif ((df[comp2] > df[comp1]) & (df[comp2] >= df[comp3]) & (df[comp2] >= df[comp4]) & (df[comp2] >= df[comp5])):
        return "'ASB Results Received' Latest/has other matching dates"
    # compares comp3 to all other comps
    elif  ((df[comp3] > df[comp1]) & (df[comp3] >= df[comp2]) & (df[comp3] >= df[comp4]) & (df[comp3] >= df[comp5])):
        return '"Arch Assessment" Latest/has other matching dates'
    # compares comp4 to all other comps
    elif ((df[comp4] > df[comp1]) & (df[comp4] >= df[comp2]) & (df[comp4] >= df[comp3]) & (df[comp4] >= df[comp5])):
        return '"Bio Assessment" Latest/has other matching dates'
    elif((df[comp5] > df[comp1]) & (df[comp5] >= df[comp2]) & (df[comp5] >= df[comp3]) & (df[comp5] >= df[comp4])):
         return '"CAC Clearance" Latest/has other matching dates'
    
    # Comp matches
    
    # All comps == "0" leave blank
    elif ((df[comp1] == "0") & (df[comp2] == "0") & (df[comp3] == "0") & (df[comp4] == "0") & (df[comp5] == "0")):
        return ""
    
    
    # All comps macth
    elif ((df[comp1] == df[comp2]) & (df[comp1] == df[comp3]) & (df[comp1] == df[comp4]) & (df[comp1] == df[comp5])):
        return "Ready for Debris Removal matches Latest  Date"
    
    # comp1 match to another other comp
    elif ((df[comp1] == df[comp2]) | (df[comp1] == df[comp3]) | (df[comp1] == df[comp4]) | (df[comp1] == df[comp5])):
        return "Ready for Debris Removal matches Latest  Date"
    else:
        return ""

In [12]:
checks.insert(checks.columns.get_loc('Ready for Debris Removal') + 1, "Max Date",
             checks.apply(find_lastest_date, comp1='Ready for Debris Removal', comp2='ASB Results Received',
                          comp3='Arch Assessment', comp4='Bio Assessment',comp5='CAC Clearance', axis=1))

In [13]:
check_style = checks.style.set_properties(**{'background-color': '#ffffb3'}, subset=["Max Date"])

In [14]:
# check_style

In [15]:
check_style.to_excel('Debris Removal Lastest Date Checks.xlsx')